In [1]:
import numpy as np
import pickle
import os

seed = 2023

In [2]:
import torch

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
from nlpsig_networks.scripts.seqsignet_functions import seqsignet_hyperparameter_search

In [4]:
output_dir = "rumours_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## Rumours

In [5]:
%run load_sbert-embeddings.py

In [6]:
df_rumours.head()

,id,label,datetime,text,timeline_id,set
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0,train
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0,train
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0,train
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0,train
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0,train


## Seq-Sig-Net

In [7]:
features = ["time_encoding", "timeline_index"]
standardise_method = ["z_score", None]
include_features_in_path = True
include_features_in_input = False

In [8]:
split_ids = torch.tensor(df_rumours['timeline_id'].astype(int))

In [9]:
num_epochs = 100
dimensions = [15]  # [50, 15]
swnu_hidden_dim_sizes_and_sig_depths = [([12], 3), ([10], 4)]
lstm_hidden_dim_sizes = [384]
ffn_hidden_dim_sizes = [[256,256],[512,512]]
dropout_rates = [0.1, 0.2]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
bidirectional = True
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5

# history_length=11

In [10]:
shift = 3
window_size = 5
n = 3

## UMAP

In [ ]:
(
    seqsignet_network_umap_11,
    best_seqsignet_network_umap_11,
    _,
    __,
) = seqsignet_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    y_data=y_data,
    output_dim=output_dim,
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["umap"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    lstm_hidden_dim_sizes=lstm_hidden_dim_sizes,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=True,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    features=features,
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    include_features_in_input=include_features_in_input,
    split_ids=split_ids,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/seqsignet_umap_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    verbose=False,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
given shift 3, window size 5 and n 3: history length = 11
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
seqsignet_network_umap_11

In [ ]:
best_seqsignet_network_umap_11["f1"].mean()

In [ ]:
best_seqsignet_network_umap_11["precision"].mean()

In [ ]:
best_seqsignet_network_umap_11["recall"].mean()

In [ ]:
np.stack(best_seqsignet_network_umap_11["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_network_umap_11["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_network_umap_11["recall_scores"]).mean(axis=0)

## GRP

In [ ]:
(
    seqsignet_network_grp_11,
    best_seqsignet_network_grp_11,
    _,
    __,
) = seqsignet_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    y_data=y_data,
    output_dim=output_dim,
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["gaussian_random_projection"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    lstm_hidden_dim_sizes=lstm_hidden_dim_sizes,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=True,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    features=features,
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    include_features_in_input=include_features_in_input,
    split_ids=split_ids,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/seqsignet_grp_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    verbose=False,
)

In [ ]:
seqsignet_network_grp_11

In [ ]:
best_seqsignet_network_grp_11["f1"].mean()

In [ ]:
best_seqsignet_network_grp_11["precision"].mean()

In [ ]:
best_seqsignet_network_grp_11["recall"].mean()

In [ ]:
np.stack(best_seqsignet_network_grp_11["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_network_grp_11["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_network_grp_11["recall_scores"]).mean(axis=0)

# history_length=20

In [ ]:
shift = 3
window_size = 5
n = 6

## UMAP

In [ ]:
(
    seqsignet_network_umap_20,
    best_seqsignet_network_umap_20,
    _,
    __,
) = seqsignet_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    y_data=y_data,
    output_dim=output_dim,
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["umap"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    lstm_hidden_dim_sizes=lstm_hidden_dim_sizes,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=True,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    features=features,
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    include_features_in_input=include_features_in_input,
    split_ids=split_ids,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/seqsignet_umap_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    verbose=False,
)

In [ ]:
seqsignet_network_umap_20

In [ ]:
best_seqsignet_network_umap_20["f1"].mean()

In [ ]:
best_seqsignet_network_umap_20["precision"].mean()

In [ ]:
best_seqsignet_network_umap_20["recall"].mean()

In [ ]:
np.stack(best_seqsignet_network_umap_20["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_network_umap_20["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_network_umap_20["recall_scores"]).mean(axis=0)

## GRP

In [ ]:
(
    seqsignet_network_grp_20,
    best_seqsignet_network_grp_20,
    _,
    __,
) = seqsignet_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    y_data=y_data,
    output_dim=output_dim,
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["gaussian_random_projection"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    lstm_hidden_dim_sizes=lstm_hidden_dim_sizes,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=True,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    features=features,
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    include_features_in_input=include_features_in_input,
    split_ids=split_ids,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/seqsignet_grp_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    verbose=False,
)

In [ ]:
seqsignet_network_grp_20

In [ ]:
best_seqsignet_network_grp_20["f1"].mean()

In [ ]:
best_seqsignet_network_grp_20["precision"].mean()

In [ ]:
best_seqsignet_network_grp_20["recall"].mean()

In [ ]:
np.stack(best_seqsignet_network_grp_20["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_network_grp_20["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_network_grp_20["recall_scores"]).mean(axis=0)

# history_length=35

In [ ]:
shift = 3
window_size = 5
n = 11

## UMAP

In [ ]:
(
    seqsignet_network_umap_35,
    best_seqsignet_network_umap_35,
    _,
    __,
) = seqsignet_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    y_data=y_data,
    output_dim=output_dim,
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["umap"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    lstm_hidden_dim_sizes=lstm_hidden_dim_sizes,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=True,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    features=features,
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    include_features_in_input=include_features_in_input,
    split_ids=split_ids,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/seqsignet_umap_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    verbose=False,
)

In [ ]:
seqsignet_network_umap_35

In [ ]:
best_seqsignet_network_umap_35["f1"].mean()

In [ ]:
best_seqsignet_network_umap_35["precision"].mean()

In [ ]:
best_seqsignet_network_umap_35["recall"].mean()

In [ ]:
np.stack(best_seqsignet_network_umap_35["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_network_umap_35["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_network_umap_35["recall_scores"]).mean(axis=0)

## GRP

In [ ]:
(
    seqsignet_network_grp_35,
    best_seqsignet_network_grp_35,
    _,
    __,
) = seqsignet_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    y_data=y_data,
    output_dim=output_dim,
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["gaussian_random_projection"],
    dimensions=dimensions,
    log_signature=True,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    lstm_hidden_dim_sizes=lstm_hidden_dim_sizes,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=True,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    features=features,
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    include_features_in_input=include_features_in_input,
    split_ids=split_ids,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/seqsignet_grp_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    verbose=False,
)

In [ ]:
seqsignet_network_grp_35

In [ ]:
best_seqsignet_network_grp_35["f1"].mean()

In [ ]:
best_seqsignet_network_grp_35["precision"].mean()

In [ ]:
best_seqsignet_network_grp_35["recall"].mean()

In [ ]:
np.stack(best_seqsignet_network_grp_35["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_network_grp_35["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_network_grp_35["recall_scores"]).mean(axis=0)